In [1]:
!pip install torch numpy


  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached torch-2.6.0-cp312-cp312-win_amd64.whl (204.1 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2


In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Sample text data
text = "deep learning is amazing. text generation using lstm networks is powerful."

# Unique characters
chars = sorted(list(set(text)))
char2idx = {char: idx for idx, char in enumerate(chars)}
idx2char = {idx: char for char, idx in char2idx.items()}

vocab_size = len(chars)
seq_length = 40
hidden_size = 128
num_layers = 2
lr = 0.003
num_epochs = 300

# Prepare dataset
data = [char2idx[ch] for ch in text]

def get_batches(data, seq_length):
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+1:i+seq_length+1]
        yield torch.tensor(x).unsqueeze(0), torch.tensor(y).unsqueeze(0)

# Define the LSTM model
class CharLSTM(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers):
        super(CharLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size=1):
        return (
            torch.zeros(num_layers, batch_size, hidden_size),
            torch.zeros(num_layers, batch_size, hidden_size)
        )

# Initialize model
model = CharLSTM(vocab_size, hidden_size, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Training loop
for epoch in range(num_epochs):
    hidden = model.init_hidden()
    total_loss = 0

    for x, y in get_batches(data, seq_length):
        optimizer.zero_grad()
        output, hidden = model(x, hidden)
        loss = criterion(output.view(-1, vocab_size), y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        hidden = tuple([h.detach() for h in hidden])

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")

# Generate text
def generate_text(start_char='d', length=200):
    model.eval()
    input = torch.tensor([[char2idx[start_char]]])
    hidden = model.init_hidden()
    result = [start_char]

    for _ in range(length):
        output, hidden = model(input, hidden)
        probs = torch.softmax(output[0, -1], dim=0).detach().numpy()
        next_idx = np.random.choice(vocab_size, p=probs)
        next_char = idx2char[next_idx]
        result.append(next_char)
        input = torch.tensor([[next_idx]])

    return ''.join(result)

# 🔥 Sample output
print("\nGenerated Text:\n")
print(generate_text('d', 300))


Epoch 50/300, Loss: 0.2343
Epoch 100/300, Loss: 0.2205
Epoch 150/300, Loss: 0.2145
Epoch 200/300, Loss: 0.2117
Epoch 250/300, Loss: 0.2101
